In [1]:
import pandas as pd

train_ratings = pd.read_csv("train_ratings.csv")
val_ratings = pd.read_csv("val_ratings.csv")
combined_ratings = train_ratings.append(val_ratings, ignore_index=True, sort=True) #cross validation is used, its ok to combine
test_ratings = pd.read_csv("test_ratings.csv")

In [2]:
from fastai.collab import * #https://docs.fast.ai/install.html
#https://docs.fast.ai/collab.html
data = CollabDataBunch.from_df(ratings=combined_ratings, user_name='userId', item_name='movieId', rating_name='rating', test=test_ratings.drop('Id', axis=1))
learn = collab_learner(data, n_factors=50, y_range=(0.5,5.0))
# learn.lr_find()
# learn.recorder.plot()
learn.fit(epochs=5, lr=1e-3, wd=1e-5)
#learn.fit_one_cycle(5, 1e-2, wd=1e-5)

epoch,train_loss,valid_loss
1,0.709542,0.670500
2,0.616901,0.656927
3,0.566475,0.667041
4,0.527403,0.680443
5,0.527079,0.692519


In [3]:
#https://docs.fast.ai/basic_train.html#Learner.predict
predictions = learn.get_preds(ds_type=DatasetType.Test)

In [5]:
pred_y = np.array(predictions[0])
pred_df_y = pd.DataFrame(pred_y, columns=['rating'])
result_y = pd.concat([test_ratings["Id"], pred_df_y], axis=1, sort=False)
result_y.to_csv('fastai_label_result.csv', columns=['Id', 'rating'], index = None)